In [1]:
import sys
import time
import pandas as pd
import numpy as np

# QCGPU Imports
import qcgpu

# ProjectQ Imports
from projectq import MainEngine
import projectq.ops as ops
from projectq.backends import Simulator
from projectq.types import Qureg

# Qiskit Imports
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute

In [2]:
def bench_qcgpu(n, depth):
    state = qcgpu.State(n)

    h = qcgpu.gate.h()
    x = qcgpu.gate.x()
    t = qcgpu.gate.t()

    start = time.time()

    for level in range(depth):
        for q in range(n):
    
            state.apply_gate(h, q)
            state.apply_gate(t, q)

            if q != 0:
                state.apply_controlled_gate(x, q, 0)
        
    runtime = time.time() - start
    return runtime
    
def bench_projectq(n, depth):
    eng = MainEngine(backend=Simulator(gate_fusion=True), engine_list=[])
    qbits = eng.allocate_qureg(n)

    start = time.time()

    for level in range(depth):
        for q in qbits:
            ops.H | q
            ops.T | q
            if q != qbits[0]:
                ops.CNOT | (q, qbits[0])

    runtime = time.time() - start

    for q in qbits:
        ops.Measure | q
    eng.flush()
    return runtime

def bench_qiskit(n, depth):
    q = QuantumRegister(n)
    c = ClassicalRegister(n)
    qc = QuantumCircuit(q, c)

    
    
    for level in range(depth):
        for i in range(n):
            qc.h(q[i])
            qc.t(q[i])

            if i != 0:
                qc.cx(q[i], q[0])

    start = time.time()
    
    job_sim = execute(qc, "local_statevector_simulator")
    job_sim.result()
    
    runtime = time.time() - start
    return runtime


In [3]:
bench_qcgpu(20,3)

0.016415119171142578

In [4]:
bench_projectq(20,3)

0.20844078063964844

In [5]:
bench_qiskit(20,3)

2.865260124206543